# Part II Getting our Audio Features for Top100 Hotlist

Goal: Generate a 5k list with diff audio features from diff authors based on our hot list 

1. Start with only one song and figure out which query etc. (done) 

2. Get Several Authors and their albums 
3. Feed quere with all album and / or track ids 

## Installing Spotipy

In order to use the Spotify API (SpotiPy) we will have to create an account in Spotify and follow these steps. Once we have done it we will start initializing the API and look at the search method for which we can introduce a "query" q, in this example we will try it with Lady Gaga:

In [ ]:
#!pip install spotipy

## Loading credentials from another config file

In [1]:
import config
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

#client_id = "5622c0f1f94243ffa051cfbdf4734578"
#client_secret = "5622c0f1f94243ffa051cfbdf4734578"

#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

## Load data file 

In [ ]:
#df_input = pd.read_csv ("../web_scraping_Marcus_Anne/list_2000.csv") #'Path where the CSV file is stored\df.csv')
#df_input.head(14)

In [ ]:
df_input["Song Artist"].value_counts()

In [ ]:
382*10

In [ ]:
#simple way

artist = "Prince"

#song = sp.search(q="prince", limit=10) 
song = sp.search(q=artist, limit=5) 

#pprint.pprint(song['tracks']['items'][0]['uri'])
song["tracks"]["items"][0]["uri"]
sp.audio_features(song["tracks"]["items"][0]["uri"])[0]

In [ ]:
#transform into a dataframe 
my_dict = sp.audio_features(song["tracks"]["items"][0]["uri"])[0] # you can provide a list of uri's

print(my_dict)
my_dict_new = { key: [my_dict[key]] for key in list(my_dict.keys()) }

#my_dict_new['name'] = [song["tracks"]["items"][0]['name']]
print(my_dict_new)

pd.DataFrame(my_dict_new)

### Test 

In [2]:
from tqdm import tqdm_notebook

In [3]:
def artist_tracks(artists):
    
    '''
    Takes a list of artist names, iterates through their Spotify albums, checks for 
    duplicate albums, then appends all the tracks in those albums to a list of lists
    '''
    
    # Each list in this list will be a track and its features
    tracks = []
    
    
    for artist in tqdm_notebook(artists):
        
        # Get the artist URI (a unique ID)
        artist_uri = sp.search(artist)['tracks']['items'][0]['artists'][0]['uri']

        # Spotify has a lot of duplicate albums, but we'll cross-reference them with this list to avoid extra loops
        album_checker = []
        
        # The starting point of our loop of albums for those artists with more than 50
        n = 0
        
        # Note the album_type = 'album'. This discounts singles, compilations and collaborations
        while len(sp.artist_albums(artist_uri, album_type = 'album', limit=50, offset = n)['items']) > 0:
            
            # Avoid overloading Spotify with requests by assigning the list of album dictionaries to a variable
            dict_list = sp.artist_albums(artist_uri, album_type = 'album', limit=50, offset = n)['items']
            
            for i, album in tqdm_notebook(enumerate(dict_list)):

                # Add the featured artists for the album in question to the checklist
                check_this_album = [j['name'] for j in dict_list[i]['artists']]
                # And the album name
                check_this_album.append(dict_list[i]['name'])
                # And its date
                check_this_album.append(dict_list[i]['release_date'])

                # Only continue looping if that album isn't in the checklist
                if check_this_album not in album_checker:
                    
                    # Add this album to the checker
                    album_checker.append(check_this_album)
                    # For every song on the album, get its descriptors and features in a list and add to the tracklist
                    tracks.extend([[artist, album['name'], album['uri'], song['name'],

                      album['release_date']] + list(sp.audio_features(song['uri'])[0].values()) 
                                   for song in sp.album_tracks(album['uri'])['items']])
            
            # Go through the next 50 albums (otherwise we'll get an infinite while loop)
            n += 50

    return tracks

In [4]:
tracks_list = ['Ciara ',
'Plain White Ts',
'Counting Crows',
'Kid Rock',
'Lady Gaga',
'U2',
'Jack Johnson',
'Jason Aldean',
'Jamie Foxx ',
'Rascal Flatts',
'Black Eyed Peas',
'N Sync',
'Mr. C The Slide Man',
'Zac Brown Band',
'Fergie',
'Norah Jones',
'Soulja Boy',
'Cupid',
'Baby Bash ',
'Jay Sean ']

In [5]:
tracks_test = artist_tracks(tracks_list) 

<ipython-input-3-99c3e4f8f852>:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for artist in tqdm_notebook(artists):


  0%|          | 0/20 [00:00<?, ?it/s]

<ipython-input-3-99c3e4f8f852>:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, album in tqdm_notebook(enumerate(dict_list)):


0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [6]:
def df_tracks(tracks_test):
    
    '''
    Takes the output of artist_tracks (i.e. a list of lists),
    puts it in a dataframe and formats it.
    '''

    df = pd.DataFrame(tracks_test, columns=['artist',
     'album_name',
     'album_uri',
     'track',
     'release_date'] + list(sp.audio_features('7tr2za8SQg2CI8EDgrdtNl')[0].keys()))

    df.rename(columns={'uri':'song_uri'}, inplace=True)

    df.drop_duplicates(subset=['artist', 'track', 'release_date'], inplace=True)

    # Reorder the cols to have identifiers first, auditory features last
    cols = ['artist', 'album_name', 'album_uri', 'track', 'release_date', 'id', 'song_uri', 'track_href',
     'analysis_url', 'type', 'danceability', 'energy', 'key',  'loudness', 'mode', 'speechiness',
     'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']

    df = df[cols]
    
    return df

In [7]:
new = df_tracks(tracks_test)


In [8]:
new

,artist,album_name,album_uri,track,release_date,id,song_uri,track_href,analysis_url,type,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Ciara,Beauty Marks,spotify:album:1Ww0yZ7iXbzkE5duwnMVNI,I Love Myself (feat. Macklemore),2019-05-10,2BlIvqI6iRD69wmbWEWJBQ,spotify:track:2BlIvqI6iRD69wmbWEWJBQ,https://api.spotify.com/v1/tracks/2BlIvqI6iRD6...,https://api.spotify.com/v1/audio-analysis/2BlI...,audio_features,...,-4.345,1,0.2930,0.2240,0.000000,0.1370,0.563,79.970,328497,4
1,Ciara,Beauty Marks,spotify:album:1Ww0yZ7iXbzkE5duwnMVNI,Level Up,2019-05-10,7ssEhls1ZYEGtBsoovkMSe,spotify:track:7ssEhls1ZYEGtBsoovkMSe,https://api.spotify.com/v1/tracks/7ssEhls1ZYEG...,https://api.spotify.com/v1/audio-analysis/7ssE...,audio_features,...,-4.666,1,0.1690,0.0142,0.010700,0.1210,0.584,152.914,204150,4
2,Ciara,Beauty Marks,spotify:album:1Ww0yZ7iXbzkE5duwnMVNI,Set,2019-05-10,0RdtNKSNJ3mkg7W988BjGN,spotify:track:0RdtNKSNJ3mkg7W988BjGN,https://api.spotify.com/v1/tracks/0RdtNKSNJ3mk...,https://api.spotify.com/v1/audio-analysis/0Rdt...,audio_features,...,-4.386,0,0.1960,0.0663,0.000269,0.8730,0.677,80.993,176848,4
3,Ciara,Beauty Marks,spotify:album:1Ww0yZ7iXbzkE5duwnMVNI,Thinkin Bout You,2019-05-10,594trlRjNdTyYLwIlcDaJM,spotify:track:594trlRjNdTyYLwIlcDaJM,https://api.spotify.com/v1/tracks/594trlRjNdTy...,https://api.spotify.com/v1/audio-analysis/594t...,audio_features,...,-4.375,0,0.0437,0.1210,0.007500,0.0939,0.819,106.988,228560,4
4,Ciara,Beauty Marks,spotify:album:1Ww0yZ7iXbzkE5duwnMVNI,Trust Myself,2019-05-10,3ZJRRdGN21Y9b7v9ajza3f,spotify:track:3ZJRRdGN21Y9b7v9ajza3f,https://api.spotify.com/v1/tracks/3ZJRRdGN21Y9...,https://api.spotify.com/v1/audio-analysis/3ZJR...,audio_features,...,-6.187,0,0.3700,0.0484,0.000005,0.2590,0.550,76.975,218035,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5594,Jay Sean,Me Against Myself,spotify:album:4IIayfG1H22HytlpNG6cwC,Man's World (Ramta Jogi),2004-01-01,1UherBzInfiGPbas2ysF2o,spotify:track:1UherBzInfiGPbas2ysF2o,https://api.spotify.com/v1/tracks/1UherBzInfiG...,https://api.spotify.com/v1/audio-analysis/1Uhe...,audio_features,...,-6.551,1,0.1270,0.0121,0.000000,0.0792,0.648,100.983,224373,4
5595,Jay Sean,Me Against Myself,spotify:album:4IIayfG1H22HytlpNG6cwC,I Believe In You,2004-01-01,4CIzMbqzUbxMD4dBDFSFnN,spotify:track:4CIzMbqzUbxMD4dBDFSFnN,https://api.spotify.com/v1/tracks/4CIzMbqzUbxM...,https://api.spotify.com/v1/audio-analysis/4CIz...,audio_features,...,-5.124,1,0.0335,0.1040,0.000000,0.2850,0.720,96.040,202867,4
5596,Jay Sean,Me Against Myself,spotify:album:4IIayfG1H22HytlpNG6cwC,One Minute,2004-01-01,3MQqY6OS9aIXzU7X9ZC1aB,spotify:track:3MQqY6OS9aIXzU7X9ZC1aB,https://api.spotify.com/v1/tracks/3MQqY6OS9aIX...,https://api.spotify.com/v1/audio-analysis/3MQq...,audio_features,...,-4.847,1,0.4830,0.0789,0.000000,0.0668,0.718,198.107,244040,4
5597,Jay Sean,Me Against Myself,spotify:album:4IIayfG1H22HytlpNG6cwC,Meri Jaan,2004-01-01,4FeLjxjO8BkJ46CsUaTSI7,spotify:track:4FeLjxjO8BkJ46CsUaTSI7,https://api.spotify.com/v1/tracks/4FeLjxjO8BkJ...,https://api.spotify.com/v1/audio-analysis/4FeL...,audio_features,...,-5.282,0,0.1230,0.0109,0.000000,0.0727,0.523,97.915,190893,4


In [9]:
new.to_csv('check_output_2010_twenty.csv')

## Starting with Spotify API

In [ ]:
# The "sp" variable has two useful funtions:
# The first usefull function is:
# .search(q='',limit=n)
# .search(q="track:"+song_name+" artist:"+artist_name,limit=5) to restrict to a song name and artist.
# Where the "q" keyword is the query you want to perform on spotify: song_name, artist,...
# while The "limit" keyword will limit the number of returned results.
#
# The second usefull function is:
# .audio_features([URL|URI|ID])
# which returns some 'features of the song', that after cleanup, we can use in order to characterize a song.

results = sp.search(q='Billie Eilish', limit=5,market="GB")
results
#json_results = json.dumps(results, ensure_ascii=True)
#json_results
#results['tracks']['items'][0]['album']

{'spotify': 'https://open.spotify.com/track/4O2N861eOnF9q8EtpH8IJu'}

In [ ]:
type(results)

In [ ]:
results.keys()

In [ ]:
results['tracks'].keys()

In [ ]:
import pprint

pprint.pprint(results)

In [ ]:
results['tracks']['items']

In [ ]:
len(results['tracks']['items'])

In [ ]:
results['tracks']['items'][0]

In [ ]:
results['tracks']['items'][0].keys()

In [ ]:
results['tracks']['items'][0]['id']

In [ ]:
for item in results['tracks']['items']:
    print("The name of song is: '{}' and the id is: {}".format(item['name'],item['external_urls']['spotify']))

In [ ]:
results["tracks"]["items"][0]['name']

In [ ]:
import pandas as pd

song = sp.search(q="Bad Guy", limit=1) 
#pprint.pprint(song['tracks']['items'][0]['uri'])
song["tracks"]["items"][0]["uri"]
sp.audio_features(song["tracks"]["items"][0]["uri"])[0]


In [ ]:
my_dict = sp.audio_features(song["tracks"]["items"][0]["uri"])[0] # you can provide a list of uri's

print(my_dict)
my_dict_new = { key: [my_dict[key]] for key in list(my_dict.keys()) }

#my_dict_new['name'] = [song["tracks"]["items"][0]['name']]
print(my_dict_new)

pd.DataFrame(my_dict_new)

Let's search for songs of several artists

In [ ]:
my_artists = ['Placebo','Marylin Manson','Janis Joplin']

my_dict = {}
#for artist in my_artists:
#    #result = sp.search(q=artist, limit= 50)
#    my_dict[artist] = sp.search(q=artist, limit= 50)

my_dict = {artist: sp.search(q=artist, limit= 50,market="GB") for artist in my_artists}
#my_dict
my_dict.keys()

In [ ]:
my_dict

In [ ]:
len(my_dict['Janis Joplin']['tracks']['items'])

In [ ]:
type(my_dict['Janis Joplin']['tracks']['items'][0])

In [ ]:
my_dict['Janis Joplin']['tracks']['items'][0].keys()

# Understanding the json

Understanding the hierachy of a JSON can make you mad. Therefore you can cosider using some online pages where you can paste your JSON file and see the "tree" structure of the file.

https://codebeautify.org/jsonviewer

So, copy the json output from the previous query and paste it on the website's left panel. On the right panel you will be able to see the hierachy of the json file.

Let's get used to the json at hand.

In [ ]:
print("The json file has the following keys: ",list(results.keys())) # We can see that we only have tracks
print("The 'tracks' key has the following child keys: ",list(results["tracks"].keys())) # Let's check the values
print("The query we made is: ",results["tracks"]["href"]) # Query we have searched 
print("The song's info is contained in: ",results["tracks"]["items"]) #items (actual tracks)
print("The limit of the query we've made is: ",results["tracks"]["limit"]) #Limit we have chosen
print("The next page if any: ",results["tracks"]["next"]) #link to the next page (next 50 tracks)
print("The starting webpage: ",results["tracks"]["offset"]) # Actual offset (starting point)
print("Starting webpage: ",results["tracks"]["previous"]) #Previous search
print("Total number of results: ",results["tracks"]["total"]) # Number of matches

## Exploring the tracks

In [ ]:
print("Total number of tracks: ",len(results["tracks"]["items"])) # 50 Tracks (as limited, it is the maximum)
print("Info of the first song: ",results["tracks"]["items"][0]) # Explore the first song 
print("The keys of the dictionary are: ",list(results["tracks"]["items"][0].keys())) # We will focus on album, artists, id, name, popularity, type and uri

## Checking albums

In [ ]:
print(results["tracks"]["items"][0]["album"]) # we have more info about the album
print(list(results["tracks"]["items"][0]["album"].keys())) # Will check artists, id, name, release date, total tracks 
print(results["tracks"]["items"][0]["album"]["artists"]) # List with artists and information
print("The album ID is: ",results["tracks"]["items"][0]["album"]["id"]) # Album ID 
print(results["tracks"]["items"][0]["album"]["name"]) # Album name (if its a single u'll get the name of the song)
print(results["tracks"]["items"][0]["album"]["release_date"]) #date in YYYY-MM-DD format
print("Songs in the album: ",results["tracks"]["items"][0]["album"]["total_tracks"]) #songs in the album

## Other info

In [ ]:
results["tracks"]["items"][0]["artists"] # Track artists
results["tracks"]["items"][0]["id"] # Track ID
results["tracks"]["items"][0]["name"] # Track name
results["tracks"]["items"][0]["popularity"] # Popularity index
results["tracks"]["items"][0]["uri"] # Basically ID

# Exploring songs

## Searching a playlist

In [ ]:
playlist = sp.user_playlist_tracks("spotify", "4rnleEAOdmFAbRcNCgZMpY")

In [ ]:
type(playlist)

In [ ]:
print(list(playlist.keys())) # Let's look at items and total:
print("Total number of songs in the playlist: ",playlist["total"]) # 4Ksongs!! Let's check items:
len(playlist["items"]) # It is limited to 100 tracks, we will have to fix it:

## Extracting the songs of a playlist

Pagination using "next"
When you collect songs from a playlist using sp.playlist_tracks, you're limited by the limit parameter, which has a maximum (and default) value of 100. When the playlist has more than 100 songs, you have to collect them by navigating through the "pages" of the results.

The parameter offset allows you to retrieve resuls starting at a certain position: if you start at position 101, you'd get the next "page" of results. An offset of 201 would give you the third page, and so on.

The function sp.next() does the same, but in a simpler way: it can be used on the results from any request to directly retrieve the results for the next page.

We can check whether there's a next page or not by accessing the key next on the results from any request.

In [ ]:
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id,market="GB")
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

get_playlist_tracks("spotify", "37i9dQZF1DX6cg4h2PoN9y")


## Getting the artists of the playlist 

In [ ]:
def get_artists_from_track(track):
    return [artist["name"] for artist in track["artists"]]

In [ ]:
def get_artists_from_playlist(playlist_id):
    tracks_from_playlist = get_playlist_tracks("spotify", playlist_id)
    return list(set(artist for subset in [get_artists_from_track(track["track"]) for track in tracks_from_playlist] for artist in subset))

In [ ]:
get_artists_from_playlist("4rnleEAOdmFAbRcNCgZMpY")

## Artists ID

In [ ]:
def get_artists_from_track(track):
    return[artist["name"] for artist in track["artists"]]

#--------------------------------------------------------------------

def get_artists_ids_from_track(track):
    return [artist["id"] for artist in track["artists"]]

In [ ]:
def get_artists_ids_from_playlist(playlist_id):
    tracks_from_playlist = get_playlist_tracks("spotify", playlist_id)
    return list(set(artist for subset in [get_artists_ids_from_track(track["track"]) for track in tracks_from_playlist] for artist in subset))

In [ ]:
artists = get_artists_from_playlist("4rnleEAOdmFAbRcNCgZMpY") # Apply the function
artists_ids = get_artists_ids_from_playlist("4rnleEAOdmFAbRcNCgZMpY")
len(artists)
len(artists_ids) # We might have more ids due to artists having the same name

# Getting the info

In this section we will work with albums to extract information. We will start by extracting all the albums of an artist.

In [ ]:
def get_albums_from_artist(artist_id):
    results = sp.artist_albums(artist_id, limit = 50,country="GB")
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

# Same for albums ids
def get_album_ids_from_artist(artist_id):
    results = sp.artist_albums(artist_id, limit = 50)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return [track["id"] for track in tracks]

In [ ]:
get_albums_from_artist("6qqNVTkY8uBg9cP3Jd7DAH")

Example: Coldplay

In [ ]:
coldplay_id = "4gzpq5DPGxSnKTe4SA8HAU"
coldplay_albums = get_albums_from_artist(coldplay_id)
coldplay_album_ids = get_album_ids_from_artist(coldplay_id)

# Check artists that played with coldplay
set([artist["name"] for track in coldplay_albums for artist in track["artists"]])

## Getting the songs of a given album

In [ ]:
def get_track_ids_from_albums(album_ids):
    return list(set([i["id"] for j in album_ids for i in sp.album(j)["tracks"]["items"]]))

In [ ]:
coldplay_songs = get_track_ids_from_albums(coldplay_album_ids)

len(coldplay_songs)